### Inference

In [4]:
# !pip install git+https://github.com/huggingface/transformers
# !pip install librosa
# !sudo apt install ffmpeg
# !pip install git+https://github.com/chuangk-p/mock-aiservice.git
# !pip install gradio 

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-9ueaigvj
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-9ueaigvj
  Resolved https://github.com/huggingface/transformers to commit 74a207404e8d4524d1fdc4aa23789694f9eef347
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9231578 sha256=d912156452728c86a0a59fd2f891425e675d5a0960c985e1d45a99b7a1605346
  Stored in directory: /tmp/pip-ephem-wheel-cache-f4mqt6te/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
Reading package lists... Done
Building d

In [5]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from mockaiservice import setting
from mockaiservice.speech import tts
import gradio as gr

2024-06-22 18:57:17.892724: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 18:57:17.892824: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 18:57:18.016205: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## ASR

In [6]:
device = 0 if torch.cuda.is_available() else "cpu"

pipe = pipeline(
    task="automatic-speech-recognition",
    model="biodatlab/whisper-th-medium-combined",
    chunk_length_s=30,
    device=device,
)

config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

In [7]:
def ASR (intput_path):
    # Perform ASR with the created pipe.
    lang = "en"
    result = pipe(intput_path, generate_kwargs={"language": lang, "task": "transcribe"}, batch_size=16)
    return result["text"]

In [7]:
# print(ASR("/kaggle/input/ai-cooking/test_asr.wav"))

/opt/conda/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 I want to know about Somdrom. Can you suggest for me?


## LLM

In [11]:
model_id = "scb10x/llama-3-typhoon-v1.5x-8b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def LLM (input_msg):
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant who're always speak Eng."},
        {"role": "user", "content": f"{input_msg}"},
    ]
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.4,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)
# 

In [141]:
# input_msg = "I want to travel in Thailand, can you recommend some places to visit and some local food to try? Make a 3 days trip plan for me."

# messages = [
#     {"role": "system", "content": "You are a helpful assistant who're always speak Eng."},
#     {"role": "user", "content": f"{input_msg}"},
# ]

In [142]:
# input_ids = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt=True,
#     return_tensors="pt"
# ).to(model.device)

# terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

# outputs = model.generate(
#     input_ids,
#     max_new_tokens=512,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.4,
#     top_p=0.9,
# )
# response = outputs[0][input_ids.shape[-1]:]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [143]:
# print(tokenizer.decode(response, skip_special_tokens=True))

Thailand is a beautiful country with a rich culture and delicious cuisine. Here's a 3-day trip plan for you, including some must-visit places and local food to try:



Day 1:

- Morning: Start your day with a visit to the Grand Palace in Bangkok, a must-see attraction in Thailand. Make sure to wear appropriate clothing, as shorts and sleeveless shirts are not allowed.

- Afternoon: Head to Wat Phra Kaew, a temple complex located within the Grand Palace, to see the famous Emerald Buddha.

- Evening: Try some local street food at Chatuchak Weekend Market, which offers a wide variety of Thai dishes. Don't miss the chance to try Pad Thai, Tom Yum soup, and Som Tam (papaya salad).



Day 2:

- Morning: Take a day trip to Ayutthaya, a UNESCO World Heritage Site located about an hour outside of Bangkok. Visit the ancient temples and ruins, including Wat Mahathat and Wat Ratchaburana.

- Afternoon: Return to Bangkok and visit the Jim Thompson House, a museum showcasing traditional Thai archite

## TTS

In [8]:
setting.set_api_key('ejjItkAMCvhD4Hr2U39B6INZt6nO5mlh')

In [9]:
def TTS (input_msg):
    tts.convert(input_msg, './output.wav')
    return "output.wav"

In [10]:
# print(TTS("I want to travel in Thailand, can you recommend some places to visit and some local food to try? Make a 3 days trip plan for me."))

output.wav


## Gradio

In [11]:
def transcribe_and_speak(audio):
    try:
        transcription = ASR(audio) #เสียงเป็นข้อความ
        # print(transcription)
        llm_output = LLM(transcription) #ข้อความเป็นคําตอบ
        tts_output = TTS(llm_output) #คําตอบเป็นเสียง
        audio_path = "output.wav"
        with open(audio_path, "wb") as f:
            f.write(tts_output["audio"])
        return transcription, audio_path
    except Exception as e:
        print(f"Error: {e}")
        return "Error in processing", None

In [12]:
interface = gr.Interface(
    fn=transcribe_and_speak,
    inputs=gr.Audio(type="filepath"),
    outputs=["text", "audio"],
)

In [13]:
interface.launch(server_name="0.0.0.0",server_port=8000)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://846e7d985ccd76c4de.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
